In [1]:
# import numpy and pandas, and DataFrame / Series
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
# Set some pandas options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)
# And some items for matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import logprocessing as lp
import io
from IPython.display import display, HTML

In [2]:
m =  lp.MetadataBuilder().load_cpu_cores(32).build()

query = {
        "header": "timestamp cpu1 io_write io_read net_down net_up memory power",
        "columns": "cpu0.timestamp, 100 - cpu0.idle, io.writet, io.readt, net.download, net.upload, memory.used, mean( energy.power_active )"
}

def dataframe_from_exp( d ):
    buffer = io.StringIO()

    p = lp.LogProcessor( 
        directory = d,
        meta = m,
        query = query,
        _file = buffer)

    p.process()
    buffer.flush()
    buffer.seek(0)
    df = pd.read_csv( buffer, delimiter=' ', index_col = False)
    buffer.close()
    
    return df
    

In [3]:
df = dataframe_from_exp("../experiments/")


In [5]:
#simple monovariable linear regression model

import statsmodels.formula.api as smf

mod = smf.ols(formula='power ~ cpu1', data=df)

res = mod.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  power   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     6354.
Date:                Wed, 07 Mar 2018   Prob (F-statistic):               0.00
Time:                        10:05:27   Log-Likelihood:                -1987.7
No. Observations:                 900   AIC:                             3979.
Df Residuals:                     898   BIC:                             3989.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     69.5805      0.474    146.881      0.0